# 03 Create Submission

## Imports

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline


## Read in Test data

In [51]:
df = pd.read_csv('../west_nile/west_nile/input/test.csv', index_col=0)

In [52]:
df.Date.max()

'2014-10-02'

## Read in X Train Data to Align Columns with submission

In [53]:
with open('../Cleansed_Data/X.pkl','rb') as f:
    X_train = pickle.load(f)

## Read in codes_df and temperature_df

In [54]:
with open('../Cleansed_Data/codes_df.pkl', 'rb') as f:
    codes_df = pickle.load(f)

In [55]:
with open('../Cleansed_Data/new_weather_df.pkl', 'rb') as f:
    new_weather_df = pickle.load(f)    

## Import Standard Scalar Object

In [56]:
with open('../Objects/ss.pkl','rb') as f:
    ss = pickle.load(f)

## Join codes_df and new_weather_df

In [57]:
df = df.join(codes_df, on='Date', rsuffix="_code_")
df = df.join(new_weather_df, on='Date')

## Extract Month from Date

In [58]:
df['Month'] = df.Date.str[5:7]

## One-hot encoding

In [59]:
df = pd.get_dummies(df, columns=['Month', 'Species'])

## Align columns with Training Dataset

In [60]:
with open('../Cleansed_Data/X_train_ss.pkl','rb') as f:
    X_train = pickle.load(f)

In [61]:
X_train.shape

(6356, 166)

In [62]:
col_to_be_added = np.setdiff1d(X_train.columns,df.columns)
col_to_be_dropped = np.setdiff1d(df.columns,X_train.columns)

df.drop(col_to_be_dropped,axis=1,inplace=True)
for col in col_to_be_added:
    df[col] = 0

In [63]:
df.loc[df[(df['Species_CULEX ERRATICUS']==0) & 
   (df['Species_CULEX PIPIENS']==0) & 
   (df['Species_CULEX PIPIENS/RESTUANS']==0) & 
   (df['Species_CULEX RESTUANS']==0) & 
   (df['Species_CULEX SALINARIUS']==0) & 
   (df['Species_CULEX TARSALIS']==0) & 
   (df['Species_CULEX TERRITANS']==0)].index, 'Species_CULEX PIPIENS'] = 1

## Scale Submission Data Using Standard Scalar

In [64]:
df_ss = pd.DataFrame(ss.transform(df), columns=df.columns, index=df.index)

## Export Submission Data

In [65]:
df_ss.to_csv('../Cleansed_Data/test_final.csv')